In [1]:
import numpy as np
import pandas as pd
import biogeme
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Constrained GBM

In [4]:
ned_data_rp = pd.read_table("Data/netherlandsRP.dat")

X = ned_data_rp[['car_cost', 'rail_cost', 'car_ivtt', 'car_walk_time', 'rail_ivtt', 'rail_acc_time', 'rail_egr_time']]
X_updt = X.copy()
X_updt['car_tt'] = X['car_ivtt'] +X['car_walk_time']
X_updt['rail_tt'] = X['rail_ivtt'] +X['rail_acc_time'] +X['rail_egr_time']

rate_G2E = 0.44378022
X_updt['car_cost'] = X_updt['car_cost'] * rate_G2E
X_updt['rail_cost'] = X_updt['rail_cost'] * rate_G2E
X_updt = X_updt.drop(columns=['car_ivtt', 'car_walk_time', 'rail_ivtt', 'rail_acc_time', 'rail_egr_time'])

y = ned_data_rp['choice']

In [5]:
X_train, X_validate, y_train, y_validate = train_test_split(X_updt, y, test_size=0.2, random_state=42)

In [42]:
xgb_constrained = xgb.XGBClassifier(#objective='binary:logistic', 
                                    #learning_rate = 0.2,
                                    n_estimators = 100, 
                                    max_depth = 1, 
                                    random_state = 1,
                                    monotone_constraints = (-1, -1, -1, -1),
                                    interaction_constraints = [["car_cost"], ["rail_cost"], ["car_tt"], ["rail_tt"]],
                                    reg_alpha = 0,
                                    reg_lambda = 0,
                                    gamma = 1
                                   )

In [43]:
xgb_constrained.get_params()

{'objective': 'binary:logistic',
 'use_label_encoder': None,
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': 1,
 'gpu_id': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': [['car_cost'],
  ['rail_cost'],
  ['car_tt'],
  ['rail_tt']],
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': 1,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': (-1, -1, -1, -1),
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'predictor': None,
 'random_state': 1,
 'reg_alpha': 0,
 'reg_lambda': 0,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbo

In [44]:
0.60451*178

107.60278

In [45]:
eval_set = [(X_train, y_train)]
xgb_constrained.fit(X_train, y_train, eval_set=eval_set)

[0]	validation_0-logloss:0.65805
[1]	validation_0-logloss:0.63799
[2]	validation_0-logloss:0.62675
[3]	validation_0-logloss:0.61956
[4]	validation_0-logloss:0.61520
[5]	validation_0-logloss:0.61200
[6]	validation_0-logloss:0.60998
[7]	validation_0-logloss:0.60974
[8]	validation_0-logloss:0.60962
[9]	validation_0-logloss:0.60956
[10]	validation_0-logloss:0.60953
[11]	validation_0-logloss:0.60952
[12]	validation_0-logloss:0.60951
[13]	validation_0-logloss:0.60951
[14]	validation_0-logloss:0.60951
[15]	validation_0-logloss:0.60951
[16]	validation_0-logloss:0.60951
[17]	validation_0-logloss:0.60951
[18]	validation_0-logloss:0.60951
[19]	validation_0-logloss:0.60951
[20]	validation_0-logloss:0.60951
[21]	validation_0-logloss:0.60951
[22]	validation_0-logloss:0.60951
[23]	validation_0-logloss:0.60951
[24]	validation_0-logloss:0.60951
[25]	validation_0-logloss:0.60951
[26]	validation_0-logloss:0.60951
[27]	validation_0-logloss:0.60951
[28]	validation_0-logloss:0.60951
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=[['car_cost'], ['rail_cost'], ['car_tt'],
                                       ['rail_tt']],
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=1,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=(-1, -1, -1, -1), n_estimators=100,
              n_jobs=None, num_parallel_tree=None, predictor=None,
              random_state=1, ...)

In [46]:
y_pred=xgb_constrained.predict(X_validate)

from sklearn.metrics import accuracy_score

accuracy_score(y_validate, y_pred)

0.5652173913043478

# Unconstrained model

In [47]:
xgb_unconstrained = xgb.XGBClassifier(#objective='binary:logistic', 
                                      learning_rate = 1,
                                      n_estimators = 1000, 
                                      max_depth = 1, 
                                      random_state = 1,
                                      reg_alpha = 0,
                                      reg_lambda = 0
                                     )

In [48]:
xgb_unconstrained.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=1, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=1, ...)

In [49]:
y_pred_unc=xgb_unconstrained.predict(X_validate)

accuracy_score(y_validate, y_pred_unc)

0.6304347826086957